# Azure OpenAI GPT-4o デモンストレーション

このNotebookでは、Azure OpenAI ServiceのGPT-4oモデルの基本的な使用方法をデモンストレーションします。

## APIの種類と使い分け

主要なAPIとして以下の3つがあります。Assistants APIはResponsesAPIに統合され、2026/08 に Deprecate されることが発表されていますので、実質  Chat Completions API と Responses API の2択になります。

### Chat Completions API
- 広く使われている“標準”の会話生成インターフェイス。
- 入力はmessagesの配列（system/developer/userなど）で、呼び出しは基本的にステートレス。
- ツール呼び出しは可能だが、会話履歴やツールの出力連携はクライアント側で管理する前提。

### Responses API
- 2025/03 に登場した次世代・統合インターフェイス
- Chat Completions の簡潔さを保ちつつ、ツール利用（web/file/computer use など）やサーバ側の状態管理（store:true/previous_response_id）を単一リクエストで扱える
- エージェント的ワークフロー（multi‑turn＋複数ツール呼び出し）に最適
- Azure AI Foundry では GA（一般提供）

### （deprecate予定）Assistants API
- 2023〜2024に提供開始された状態管理付きの“アシスタント”構築 API（Assistant/Thread/Runなどのオブジェクトで会話を保持）
- Responses API への置き換え方針が公式に示されており、2026年中頃にサンセット予定（段階的に縮退）
- エージェントサービス（Azure AI Foundry Agent Service）はこの系譜の概念・プロトコルを踏襲

ここでは、基本的なデモとして `Chat Completions API` を使用します。


## 前提条件

1. Azure OpenAI Serviceリソースがデプロイされている
2. GPT-4oモデルがデプロイされている
3. 必要なPythonパッケージがインストールされている
4. 環境変数が適切に設定されている

## 1. 環境設定と必要なライブラリのインポート

### 環境変数の設定

ここでは、必要な環境変数を.envから読み出す方式でパラメータを設定します。（テスト用の簡易方式です。）
Notebook と同じディレクトリに `.env` ファイルを作成し、次の内容を保存します。
```ini
AZURE_OPENAI_ENDPOINT=https://<your-resource>.openai.azure.com/
AZURE_OPENAI_API_KEY=<your-api-key>
AZURE_OPENAI_API_VERSION=2024-02-15-preview
GPT4O_DEPLOYMENT_NAME=gpt-4o
```

注意: API キー等の機密情報はリポジトリにコミットしないでください。既にコミットしてしまった場合は、速やかにキーをローテーションし、履歴からの除去手順を実施してください。


In [49]:
# .env テンプレートを作成する（存在しない場合のみ）
from pathlib import Path
env_path = Path('.env')
if not env_path.exists():
    env_path.write_text('AZURE_OPENAI_ENDPOINT=\nAZURE_OPENAI_API_KEY=\nAZURE_OPENAI_API_VERSION=2024-02-15-preview\nGPT4O_DEPLOYMENT_NAME=gpt-4o\n')
    print('.env テンプレートを作成しました。値を入力してください。')
else:
    print('.env は既に存在します。編集してください。')

.env は既に存在します。編集してください。


In [50]:
# 必要なライブラリのインストール
import sys
!{sys.executable} -m pip install openai python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
# .env.dev を強制パースして os.environ に直接設定する
from dotenv import dotenv_values
from pathlib import Path
import os

# ノートブックから見たリポジトリルートへの相対パス (demos/openai から ../../ でルートへ)
repo_root = Path(__file__).parent.parent.parent if '__file__' in globals() else Path.cwd().parent.parent
env_path = repo_root / '.env.dev'

if env_path.exists():
    # dotenv_values で .env.dev を辞書として読み込む
    parsed = dotenv_values(env_path)
    # 各キーを os.environ に書き込む(空文字や None はスキップ)
    for k, v in parsed.items():
        if v:
            os.environ[k] = v
    print(f'環境変数を読み込みました: {env_path}')
    print(f'{len(parsed)} 個のキーをパースし、os.environ に設定しました。')
else:
    print(f'警告: {env_path} が見つかりません。')

# 設定状況を確認
print('設定状況:')
print('  AZURE_OPENAI_ENDPOINT:', 'SET' if os.getenv('AZURE_OPENAI_ENDPOINT') else 'NOT SET')
print('  AZURE_OPENAI_API_KEY:', 'SET' if os.getenv('AZURE_OPENAI_API_KEY') else 'NOT SET')
print('  GPT4O_DEPLOYMENT_NAME:', os.getenv('GPT4O_DEPLOYMENT_NAME'))

環境変数を読み込みました: c:\Users\toohta\Gitrepos\AzureAIServicesDemo\.env.dev
13 個のキーをパースし、os.environ に設定しました。
設定状況:
  AZURE_OPENAI_ENDPOINT: SET
  AZURE_OPENAI_API_KEY: SET
  GPT4O_DEPLOYMENT_NAME: gpt-4o


In [52]:
import os
import openai
from dotenv import load_dotenv
import json
from datetime import datetime

# 環境変数の読み込み
load_dotenv()

# 設定の確認
print("=== Azure OpenAI 設定確認 ===")
endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-02-15-preview')
deployment_name = os.getenv('GPT4O_DEPLOYMENT_NAME', 'gpt-4o')

print(f"エンドポイント: {endpoint}")
print(f"APIバージョン: {api_version}")
print(f"デプロイメント名: {deployment_name}")

if not endpoint:
    print("⚠️ AZURE_OPENAI_ENDPOINT が設定されていません")
else:
    print("✅ 設定が確認されました")

=== Azure OpenAI 設定確認 ===
エンドポイント: https://aiservicesdemo-ai-dev-2ape77.cognitiveservices.azure.com/
APIバージョン: 2024-02-15-preview
デプロイメント名: gpt-4o
✅ 設定が確認されました


## 2. Azure OpenAI クライアントの初期化

In [53]:
# Azure OpenAI クライアントの初期化
try:
    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        api_version=api_version
    )
    print("✅ Azure OpenAI クライアントが正常に初期化されました")
except Exception as e:
    print(f"❌ クライアント初期化エラー: {str(e)}")
    print("APIキーが正しく設定されているか確認してください")

✅ Azure OpenAI クライアントが正常に初期化されました


## 3. 基本的なチャット機能のデモ

In [54]:
def chat_with_gpt4o(message, system_message="あなたは親切なAIアシスタントです。"):
    """
    GPT-4oとの基本的なチャット機能
    """
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": message}
            ],
            max_tokens=800,
            temperature=0.7
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

# デモ実行
test_message = "Azure AI Servicesについて簡潔に説明してください。"
print(f"ユーザー: {test_message}")
print(f"\nGPT-4o: {chat_with_gpt4o(test_message)}")

ユーザー: Azure AI Servicesについて簡潔に説明してください。

GPT-4o: Azure AI Servicesは、Microsoft Azureが提供するクラウドベースのAI機能群です。これには、機械学習、自然言語処理、画像認識、音声処理などのAI機能が含まれており、開発者が簡単に高度なAI機能をアプリケーションに統合可能です。主なサービスには、Azure Machine Learning、Cognitive Services（視覚、音声、言語、意思決定などのAPI）、およびBot Servicesなどがあります。これらを利用することで、AIソリューションを迅速かつ効率的に開発・展開できます。


## 4. ストリーミング応答のデモ

In [55]:
def stream_chat_with_gpt4o(message, system_message="あなたは親切なAIアシスタントです。"):
    """
    ストリーミング形式でのGPT-4o応答
    """
    try:
        response = client.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": message}
            ],
            max_tokens=800,
            temperature=0.7,
            stream=True
        )
        
        print("GPT-4o (ストリーミング): ", end="", flush=True)
        full_response = ""
        
        for chunk in response:
            # choices が存在し、delta に content があるかチェック
            if chunk.choices and len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if hasattr(delta, 'content') and delta.content is not None:
                    content = delta.content
                    print(content, end="", flush=True)
                    full_response += content
        
        print("\n")
        return full_response
    
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        return None

# ストリーミングデモ実行
stream_message = "Azure Cognitive Servicesの主要な機能を3つ教えてください。"
print(f"ユーザー: {stream_message}\n")
stream_response = stream_chat_with_gpt4o(stream_message)

ユーザー: Azure Cognitive Servicesの主要な機能を3つ教えてください。

GPT-4o (ストリーミング): Azure Cognitive Servicesは、Microsoftが提供するAIサービスのセットであり、開発者が高度なAI機能をアプリケーションに簡単に統合できるように設計されています。以下はその主要な機能の3つです：

### 1. **Vision（視覚）**
   - コンピュータビジョンを使って画像や動画を分析し、情報を抽出する機能です。
   - 例: 
     - 画像認識（物体や顔の検出）
     - OCR（文字認識）
     - カスタムビジョン（特定の用途にカスタマイズされたモデルの作成）
     - 画像内のタグ付け、説明生成、ランドマーク認識など。

---

### 2. **Speech（音声）**
   - 音声に関連するAI機能を提供します。
   - 例: 
     - 音声認識（Speech-to-Text）
     - 音声合成（Text-to-Speech）
     - 音声翻訳
     - カスタマイズ音声モデルの作成。
     - 会話型AIにおける音声インターフェースの構築。

---

### 3. **Language（言語）**
   - 自然言語処理（NLP）を利用してテキストを理解、分析、生成する機能です。
   - 例: 
     - テキスト分析（感情分析、キーフレーズ抽出、言語検出など）
     - 翻訳（多言語対応）
     - Q&Aサービス（ナレッジベースからの回答生成）
     - カスタム言語モデルやローカライズされた言語処理。

これらのサービスを利用することで、アプリケーションに視覚、音声、言語に関連する高度なAI機能を簡単に組み込むことが可能です。他にも「Decision（意思決定）」や「Search（検索）」などの機能も提供されていますが、上記3つが特に広く活用されています。



## 5. 異なる温度設定での応答比較

In [56]:
def compare_temperature_responses(message, temperatures=[0.0, 0.5, 1.0]):
    """
    異なる温度設定での応答を比較
    """
    print(f"質問: {message}\n")
    
    for temp in temperatures:
        try:
            response = client.chat.completions.create(
                model=deployment_name,
                messages=[
                    {"role": "system", "content": "あなたは簡潔に答えるAIアシスタントです。"},
                    {"role": "user", "content": message}
                ],
                max_tokens=200,
                temperature=temp
            )
            
            print(f"🌡️ 温度設定 {temp}:")
            print(f"{response.choices[0].message.content}\n")
            print("-" * 50)
            
        except Exception as e:
            print(f"温度 {temp} でエラー: {str(e)}\n")

# 温度比較デモ
comparison_question = "クラウドコンピューティングの利点を1つ教えてください。"
compare_temperature_responses(comparison_question)

質問: クラウドコンピューティングの利点を1つ教えてください。

🌡️ 温度設定 0.0:
柔軟性が高く、必要に応じてリソースを迅速に拡張または縮小できる点です。

--------------------------------------------------
🌡️ 温度設定 0.5:
柔軟性が高く、必要に応じてリソースを迅速に拡張・縮小できる点です。

--------------------------------------------------
🌡️ 温度設定 1.0:
柔軟性が高く、必要に応じてリソースを拡張や縮小できる点です。

--------------------------------------------------


## 6. 使用量とコストの監視

In [57]:
def get_usage_info(response):
    """
    API応答から使用量情報を取得
    """
    if hasattr(response, 'usage'):
        usage = response.usage
        print("📊 使用量情報:")
        print(f"  - プロンプトトークン: {usage.prompt_tokens}")
        print(f"  - 完了トークン: {usage.completion_tokens}")
        print(f"  - 総トークン: {usage.total_tokens}")
        return usage
    return None

# 使用量監視付きのリクエスト例
try:
    usage_response = client.chat.completions.create(
        model=deployment_name,
        messages=[
            {"role": "system", "content": "あなたは技術的なAIアシスタントです。"},
            {"role": "user", "content": "機械学習と深層学習の違いを教えてください。"}
        ],
        max_tokens=300,
        temperature=0.7
    )
    
    print("回答:")
    print(usage_response.choices[0].message.content)
    print("\n" + "=" * 50)
    get_usage_info(usage_response)
    
except Exception as e:
    print(f"エラー: {str(e)}")

回答:
機械学習（Machine Learning）と深層学習（Deep Learning）はどちらも人工知能（AI）の分野に属し、データからパターンを学習して予測や分類を行う技術ですが、そのアプローチや特徴にはいくつか違いがあります。以下に主な違いを説明します。

---

### 1. **定義と範囲**
- **機械学習（ML）:**
  機械学習は、コンピュータが明示的にプログラムされることなく、データから学習し、パターンを見つけるアルゴリズムの集合です。機械学習のアルゴリズムには、線形回帰、ロジスティック回帰、決定木、サポートベクトルマシン（SVM）、k-近傍法（k-NN）などがあります。

- **深層学習（DL）:**
  深層学習は機械学習の一分野で、人工ニューラルネットワーク（ANN）を基盤とする技術です。特に、多層構造のニューラルネットワーク（ディープニューラルネットワーク）を使用し、大量のデータから自動的に特徴を抽出して学習

📊 使用量情報:
  - プロンプトトークン: 42
  - 完了トークン: 300
  - 総トークン: 342


## 7. 対話履歴を保持するチャットセッション

In [58]:
class ChatSession:
    def __init__(self, system_message="あなたは親切で知識豊富なAIアシスタントです。"):
        self.messages = [{"role": "system", "content": system_message}]
        self.total_tokens_used = 0
    
    def send_message(self, user_message):
        # ユーザーメッセージを履歴に追加
        self.messages.append({"role": "user", "content": user_message})
        
        try:
            response = client.chat.completions.create(
                model=deployment_name,
                messages=self.messages,
                max_tokens=500,
                temperature=0.7
            )
            
            assistant_message = response.choices[0].message.content
            
            # アシスタントの応答を履歴に追加
            self.messages.append({"role": "assistant", "content": assistant_message})
            
            # 使用量を追跡
            if hasattr(response, 'usage'):
                self.total_tokens_used += response.usage.total_tokens
            
            return assistant_message
        
        except Exception as e:
            return f"エラー: {str(e)}"
    
    def get_conversation_history(self):
        return [msg for msg in self.messages if msg["role"] != "system"]
    
    def clear_history(self):
        # システムメッセージ以外をクリア
        self.messages = [self.messages[0]]
        self.total_tokens_used = 0

# チャットセッションのデモ
chat = ChatSession("あなたはAzure専門のテクニカルアドバイザーです。")

# 複数のやり取りをデモ
questions = [
    "Azure OpenAIとは何ですか？",
    "先ほどの回答で言及されたGPT-4について詳しく教えてください。",
    "コスト最適化のベストプラクティスはありますか？"
]

for i, question in enumerate(questions, 1):
    print(f"\n💬 会話 {i}:")
    print(f"ユーザー: {question}")
    answer = chat.send_message(question)
    print(f"アシスタント: {answer}")
    print("-" * 80)

print(f"\n📊 セッション総計: {chat.total_tokens_used} トークン使用")


💬 会話 1:
ユーザー: Azure OpenAIとは何ですか？
アシスタント: Azure OpenAIとは、Microsoft Azureが提供するクラウドベースのサービスで、OpenAIの高度なAIモデル（例：GPTシリーズ、Codex、DALL·Eなど）をAzureプラットフォーム上で利用できるようにしたものです。このサービスを利用することで、企業や開発者はOpenAIの強力な自然言語処理（NLP）、コード生成、画像生成などの機能を、自分のアプリケーションやサービスに統合することができます。

### 主な特徴
1. **OpenAIモデルの活用**:
   Azure OpenAIでは、OpenAIの高度なAIモデル（例：GPT-4、GPT-3.5など）を利用できます。これにより、自然言語処理や生成的AIの機能を組み込んだアプリケーション開発が可能です。

2. **Azureインフラとの統合**:
   OpenAIのAIモデルをAzureのセキュリティ、スケーラビリティ、およびコンプライアンス基盤の上で実行できます。これにより、企業レベルの信頼性やデータ管理が実現します。

3. **ユースケースの幅広さ**:
   - テキスト生成（チャットボット、コンテンツ生成）
   - コード生成（プログラミング支援）
   - 翻訳や要約
   - 画像生成（DALL·Eを活用したクリエイティブな画像生成）
   - 分析やデータ抽出など

4. **セキュリティとプライバシー**:
   データはMicrosoftのセキュリティ基準に基づいて保護され、企業は自分のデータを完全に制御できます。モデルのトレーニングや推論で使用されるデータは、OpenAIモデルのトレーニングに再利用されないことが保証されています。

5. **REST APIの提供**:
   Azure OpenAIはREST APIを介してアクセスできるため、さまざまな開発環境やプログラミング言語で簡単に統合できます。

### 利用例
- **カスタマーサポート**:
--------------------------------------------------------------------------------

💬 会話 2:
ユーザー: 先ほどの回答で言及されたG

## 8. まとめ

このNotebookでは以下の機能をデモンストレーションしました:

1. ✅ **環境設定**: Azure OpenAI の接続設定
2. ✅ **基本チャット**: 単発の質問応答
3. ✅ **ストリーミング応答**: リアルタイムでの応答表示
4. ✅ **パラメータ調整**: 温度設定による応答の違い
5. ✅ **使用量監視**: トークン消費量の追跡
6. ✅ **対話セッション**: 会話履歴を保持したチャット

### 次のステップ

- 他のAzure AI Servicesとの連携
- より高度なプロンプトエンジニアリング
- 関数呼び出し（Function Calling）の実装
- RAG（Retrieval-Augmented Generation）パターンの実装

### 参考リンク

- [Azure OpenAI Service ドキュメント](https://docs.microsoft.com/azure/cognitive-services/openai/)
- [OpenAI Python ライブラリ](https://github.com/openai/openai-python)
- [Azure AI Services](https://docs.microsoft.com/azure/cognitive-services/)
- [OpenAI 公式（Responses 移行ガイド）](https://platform.openai.com/docs/guides/migrate-to-responses)
- [Azure Learn（Responses API 概要・リージョン・モデル・既知制限）](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/responses)
- [Responses API is now generally available](https://azurefeeds.com/2025/08/28/the-responses-api-in-azure-ai-foundry-is-now-generally-available/)
- [Assistants API のデプリケーション](https://platform.openai.com/docs/deprecations/deprecations)
- [OpenAI API: Responses vs Chat Completions](https://simonwillison.net/2025/Mar/11/responses-vs-chat-completions/) 
- [Azure AI Foundry Blog: New tools in Responses](https://devblogs.microsoft.com/foundry/introducing-new-tools-and-features-in-the-responses-api-in-azure-ai-foundry/)